#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 1:  Retrieval Models: Boolean + Vector Space

### 100 points [5% of your final grade]

### Due: Thursday, February 2 by 11:59pm

*Goals of this homework:* In this homework you will get first hand experience building a text-based mini search engine. In particular, there are three main learning objectives: (i) the basics of tokenization (e.g. stemming, case-folding, etc.) and its effect on information retrieval; (ii) basics of index building and Boolean retrieval; and (iii) basics of the Vector Space model and ranked retrieval.

*Submission Instructions:* To submit your homework, rename this notebook as lastname_firstinitial_hw#.ipynb. For example, my homework submission would be: caverlee_j_hw1.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 5 at 11:59pm.

## Dataset

We provide the dataset, [southpark_scripts.zip](https://www.dropbox.com/s/6rzfsbn97s8vwof/southpark_scripts.zip), which includes scripts for episodes of the first twenty seasons of the TV show South Park. You will build an inverted index over these scripts where each episode should be treated as a single document. There are 277 episodes (documents) to index and search on.

## (20 points) Part 1: Parsing

First, you should tokenize documents using **whitespaces and punctuations as delimiters** but do not remove stop words. Your parser needs to also provide the following two confgiuration options:
* Case-folding
* Stemming: use [nltk Porter stemmer](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter)

Please note that you should stick to the stemming package listed above. Otherwise, given the same query, the results generated by your code can be different from others.

In [22]:
# configuration options
use_stemming = True # or false
use_casefolding = True # or false


In [23]:
import glob,os,re
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

#utility function to speed up stemming 
stem_cache = {}
def cache_based_stemming (token):
    if token not in stem_cache:
        stem_cache[token] = stemmer.stem(token)        
    return stem_cache[token]



all_file_content=[]  #will contain the tokens of all documents 
content = {}

# Parses all documents and tokenizes them 
for file in glob.glob("southpark_scripts/*.txt"):
    with open(file) as myfile:
        
        #'fname' will contain the 4 digit name of the document. Eg, southpark_scripts/0101.txt => fname=0101
        file = file.split('\\')   
        fname = file[-1]
        fname = re.match(r'\d{4}',fname)
        fname = fname.group()  
        
        # File reading 
        content[fname]=  myfile.read()

    content[fname] = re.split('\W+',content[fname])
    content[fname] = list(filter(None, content[fname])) 
    
    if use_stemming is True:
        for i in range(len(content[fname])):
            content[fname][i] = cache_based_stemming(content[fname][i])
    if use_casefolding is True:
        for i in range(len(content[fname])):
            content[fname][i] = content[fname][i].lower()
            
    all_file_content += content[fname]
    myfile.close()

vocabulary = list(set(all_file_content))

print("For configuration use_stemming = "+ str(use_stemming) + " and use_casefolding = "+ str(use_casefolding))
print("Unique tokens = " + str(len(vocabulary)))


For configuration use_stemming = True and use_casefolding = True
Unique tokens = 17144


### Observations

Once you have your parser working, you should report here the size of your dictionary under the four cases. That is, how many unique tokens do you have with stemming on and casefolding on? And so on. You should fill in the following

* Stemming + Casefolding       = 17144
* Stemming + No Casefolding    = 17368
* No Stemming + Casefolding    = 23806
* No Stemming + No Casefolding = 29550


## (40 points) Part 2: Boolean Retrieval

In this part you build an inverted index to support Boolean retrieval. We only require your index to  support AND queries. In other words, your index does not have to support OR, NOT, or parentheses. Also, we do not explicitly expect to see AND in queries, e.g., when we query **great again**, your search engine should treat it as **great** AND **again**.

Example queries:
* Rednecks
* Troll Trace
* Respect my authority
* Respect my authoritah
* Respected my authority

In [24]:
def BooleanRetrieval (search_text):
    query = search_text.split() 

    if use_stemming is True:
        for i in range(len(query)):
            query[i] = cache_based_stemming(query[i])

    if use_casefolding is True:
        for i in range(len(query)):
            query[i] = query[i].lower()

    inv_index_dict = {}  #placeholder for inverted index of all documents
    query = list(set(query)) #keeping unique items in query 
    for item in query:
        inv_index_dict[item]= []

    #query has all the tokens for 'search_text' 
    #'inv_index_dict' is the inverted index dictionary. 'keys' are the tokens of the 'query', 'values' are list of documnets where those 'item' are present.
    #For now, it just has a placeholder for each token of search_text, needs to be populated 

    ##--------------------------------------------------
    for file in glob.glob("southpark_scripts/*.txt"):

        file= file.split('\\')
        fname = file[-1]   
        fname = re.match(r'\d{4}',fname)
        fname = fname.group()

        #intersection of tokens between opened text file ('file') and 'search_text'
        common = set(content[fname]) & set(query)
        for item in common:
            inv_index_dict[item].append(fname)
        
        myfile.close()

   
    result = set(inv_index_dict[query[0]] )
    for item in query:
        result = result & set(inv_index_dict[item])
    
    result = [ele+'.txt' for ele in result]
    return list(result)

In [25]:
search_text = raw_input('Boolean Search:')
# search for the input using your index and print out ids of matching documents
print(BooleanRetrieval(search_text))


Boolean Search:hh hh
[]


Your observations here.

### BONUS: Phrase Queries

Your search engine needs to also (optionally) support phrase queries of arbitrary length. Use quotes in a query to tell your search engine this is a phrase query. Again, we don't explicitly type AND in queries and never use OR, NOT, or parentheses.

In [26]:
def PhraseQuerySearch(search_text):
    query = search_text.split()

    if use_stemming is True:
        for i in range(len(query)):
            query[i] = cache_based_stemming(query[i])

    if use_casefolding is True:
        for i in range(len(query)):
            query[i] = query[i].lower()

    match_doc_result = []
    ##--------------------------------------------------
    
    for file in glob.glob("southpark_scripts/*.txt"):                
        file= file.split('\\')
        fname = file[-1]
        fname = re.match(r'\d{4}',fname)
        fname = fname.group()

        for i in xrange(len(content[fname])):
            if content[fname][i] == query[0]: #if there is a match with the first token of query, then proceed to match other tokens consecutively
                k = i + 1 
                j = 1 
                while j<len(query): #matching remaining tokens 
                    if content[fname][k] == query[j]:
                        k = i+1 
                        j = j+1 
                    else:
                        break

                if j==len(query): #there is a match for this document with phrase query 
                    match_doc_result.append(fname+'.txt')
                    break
                    
        myfile.close()
                    
    return match_doc_result


In [32]:
search_text = raw_input('Boolean Search (Phrase Query:')
print(PhraseQuerySearch(search_text))
# search for the input using your index and print out ids of matching documents


Boolean Search (Phrase Query:my authority
['0304.txt', '1610.txt']


## (40 points) Part 3: Ranked Retrieval

In this part, your job is to support queries over an index that you build. This time you will use the vector space model plus cosine similarity to rank documents.

**TFIDF:** For the document vectors, use the standard TFIDF scores. That is, use the log-weighted term frequency $1+log(tf)$; and the log-weighted inverse document frequency $log(\frac{N}{df})$. For the query vector, use simple weights (the raw term frequency). For example:
* query: troll $\rightarrow$ (1)
* query: troll trace $\rightarrow$ (1, 1)

**Output:**
For a given query, you should rank all the 277 documents but you only need to output the top-5 documents (i.e. document ids) plus the cosine score of each of these documents. For example:

* result1 - score1
* result2 - score2
* result3 - score3
* result4 - score4
* result5 - score5

You can additionally assume that your queries will contain at most three words. Be sure to normalize your vectors as part of the cosine calculation!

In [34]:

def RankRetrieval (search_text):
    import glob,os,re,math,operator
    import numpy as np
    from nltk.stem import PorterStemmer
    from scipy import spatial

    
    ##------------------------------------------------------------------------------------------
    #Each element of vocabulary serves as a 'key' in dictionary and the 'value' is the list of count for all documents 
    #Eg, count_lookup_vocab['The']=[10, 20, 11, ....]   => the token 'The' appears 10 times in document in 0101.txt, 20 times in 0102.txt and so on 

    #--------------------------------------------------------------------------------------------
    #Step 1: creating empty dict 'count_lookup_vocab'
    count_lookup_vocab = {}
    
    for item in vocabulary:
        count_lookup_vocab[item] = [0 for x in xrange(277)]           #creating placeholder for 277 documents 

    #--------------------------------------------------------------------------------------------
    #Step 2: populating 'count_lookup_vocab' dictionary : preprocessing 

    cosine_score ={}  # will contain cosine scores of all documents

    file_index = -1 
    file_names=[]  # will contain name of all .txt files
    for file in sorted(glob.glob("southpark_scripts/*.txt")):
        file_index +=1 

        file  = file.split('\\')
        fname = file[-1]   
        fname = re.match(r'\d{4}',fname)
        fname = fname.group()

        for item in content[fname]:        #every 'item' in 'content' belongs to 'vocabulary'
            count_lookup_vocab[item][file_index] +=1  
        
        file_names.append(fname)
        
        cosine_score[fname] = []

    ##---------------------------------------------------------------------------------------------
    #Step 3: 'search_text' tokenization 
    N = len(file_names)
    
    temp_doc_content = []
    sorted_cosine_score ={}
    
    query = search_text.split() 

    if use_stemming is True:
        for i in range(len(query)):
            query[i] = cache_based_stemming(query[i])

    if use_casefolding is True:
        for i in range(len(query)):
            query[i] = query[i].lower()

    #-----------------------------------------------------------------------------------------------
    #Step 4: create TF-IDF score vector (doc_tfidf_vector) for each document using the count_lookup_vocab 
    #         and computing the cosine scores
    
    for doc_name in cosine_score:
        doc_number = file_names.index(doc_name) 
        
        #reseting
        doc_tfidf_vector =[]  
        query_tfidf_vector= []  
        temp_doc_content =[]
        
        temp_doc_content = list(set(content[doc_name]))
        for token in temp_doc_content:
            log_tf= 1+math.log10(count_lookup_vocab[token][doc_number])
                
            N = len(file_names)
            df = sum([1 for ele in count_lookup_vocab[token] if ele!=0])
            log_idf = math.log10(float(N)/df)
                
            doc_tfidf_vector.append(log_tf*log_idf)
            
            query_tfidf_vector.append(query.count(token))
            
        if sum(doc_tfidf_vector)==0 or sum(query_tfidf_vector)==0:
            sorted_cosine_score[doc_name+'.txt'] = 0 
        else:
            sorted_cosine_score[doc_name+'.txt'] = (1- spatial.distance.cosine(doc_tfidf_vector, query_tfidf_vector))
            
    sorted_cosine_score = sorted(sorted_cosine_score.items(), key=operator.itemgetter(1),reverse= True)
 
    return sorted_cosine_score
            
 

In [38]:
search_text = raw_input('Ranked Search:')
sorted_cosine_score =RankRetrieval(search_text)

print("Top 5 matches with respective cosine scores are- ")
for i in xrange(10):
    if sorted_cosine_score[0][1]==0:
        print("No document match for the query")
        break
    elif sorted_cosine_score[i][1]==0:
        break 
    else:
        print(sorted_cosine_score[i])


Top 5 matches with respective cosine scores are- 
('1402.txt', 0.04683562634556393)
('0904.txt', 0.046774750980709268)
('1204.txt', 0.041259329368012909)
('0304.txt', 0.039222976908189988)
('1005.txt', 0.037307013977343817)
('0503.txt', 0.036991702745555743)
('1610.txt', 0.035638042526848879)
('2004.txt', 0.034594743613401091)
('0314.txt', 0.032964087233073203)
('1907.txt', 0.032797519955782284)


## How we grade

The grader will randomly pick 5-10 queries to test your program. You are welcome to discuss the results returned by your search engine with others on Piazza.